## Escribir una forma mínima

Vamos a actualizar nuestra plantilla de detalles de encuestas («polls/detail.html») a partir del último tutorial, de modo que la plantilla contiene un elemento HTML <form>:

In [ ]:

# polls/templates/polls/detail.html

<form action="{% url 'polls:vote' question.id %}" method="post">
{% csrf_token %}
<fieldset>
    <legend><h1>{{ question.question_text }}</h1></legend>
    {% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}
    {% for choice in question.choice_set.all %}
        <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}">
        <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br>
    {% endfor %}
</fieldset>
<input type="submit" value="Vote">
</form>



Un resumen rápido:

    - La plantilla anterior muestra un botón de opción para cada pregunta de selección. El valor de cada botón de opción es la ID de la pregunta de selección asociada. El nombre de cada botón de opción es "choice". Esto significa que cuando alguien selecciona uno de los botones de opción y envía el formulario, envíe los datos POST Choice=# donde # es la ID de la opción seleccionada. Este es el concepto básico de los formularios HTML.
    
    - Establecemos la acción del formulario en {% url 'polls:vote' question.id %} y configuramos método="post". Usar método="post" (a diferencia de método="get") es muy importante, porque el acto de enviar este formulario alterará los datos del lado del servidor. Siempre que cree un formulario que altere los datos del lado del servidor, utilice el método="post". Este consejo no es específico de Django; es una buena práctica de desarrollo web en general.
    
    - ``forloop.counter`` indica cuantas veces la etiqueta ha pasado por el bucle
    
    - Dado que estamos creando un formulario POST (que puede tener el efecto de modificar datos), debemos preocuparnos por las falsificaciones de solicitudes entre sitios. Afortunadamente, no tienes que preocuparte demasiado, porque Django viene con un sistema útil para protegerte contra esto. En resumen, todos los formularios POST dirigidos a URL internas deben utilizar la etiqueta de plantilla {% csrf_token %}.

A continuación, vamos a crear una vista Django que maneje los datos enviados y haga algo con ello. Recuerde que en el Tutorial 3, creamos una URLconf para la aplicación de encuestas que incluye esta línea:

También creamos una implementación simulada de la función vote(). Vamos a crear una versión real. Agregue lo siguiente a polls/views.py:

In [ ]:

# polls/views.py

from django.db.models import F
from django.http import HttpResponse, HttpResponseRedirect
from django.shortcuts import get_object_or_404, render
from django.urls import reverse

from .models import Choice, Question


# ...
def vote(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        # Redisplay the question voting form.
        return render(
            request,
            "polls/detail.html",
            {
                "question": question,
                "error_message": "You didn't select a choice.",
            },
        )
    else:
        selected_choice.votes = F("votes") + 1
        selected_choice.save()
        # Always return an HttpResponseRedirect after successfully dealing
        # with POST data. This prevents data from being posted twice if a
        # user hits the Back button.
        return HttpResponseRedirect(reverse("polls:results", args=(question.id,)))



Este código incluye algunas cosas que no hemos cubierto todavía en este tutorial:

    - request.POST es un objeto tipo diccionario que le permite acceder a los datos presentados mediante el nombre clave. En este caso, request.POST['choice'] retorna la ID de la opción seleccionada como una cadena. Los valores request.POST son siempre cadenas.

    - Tenga en cuenta que Django también proporciona request.GET para acceder a datos GET de la misma manera, pero estamos usando de manera explícita request.POST en nuestro código, para garantizar que los datos sólo se modifiquen a través de una llamada POST.

    - request.POST['choice'] levantará un excepción :exc: KeyError si choice no fue proporcionado en los datos POST. El código anterior busca la excepción KeyError y muestra de nuevo el formulario de pregunta con un mensaje de error si no se ha proporcionado choice.

    - F("votos") + 1 indica a la base de datos que aumente el recuento de votos en 1.

    - Después de incrementar el conteo de la elección, el código retorna una HttpResponseRedirect en lugar de una HttpResponse normal HttpResponseRedirect toma un único argumento: La URL a la que el usuario será redirigido (vea el siguiente aspecto de cómo construimos la URL en este caso).

    - Como señala el comentario anterior de Python, siempre debe devolver un HttpResponseRedirect después de tratar con éxito los datos POST. Este consejo no es específico de Django; es una buena práctica de desarrollo web en general.

    - Estamos utilizando la función reverse() en el constructor HttpResponseRedirect en este ejemplo. Esta función ayuda a evitar tener que codificar una URL en la función de vista. Se proporciona el nombre de la vista a la que queremos pasar el control y la parte de la variable del patrón de URL que señala esa vista. En este caso, utilizando la URLconf que configuramos en el Tutorial 3, esta llamada reverse() retornará una cadena como

    donde el 3 es el valor de question.id. Esta URL redirigida entonces llamará a la vista 'results' para mostrar la página final.

Como se mencionó en el Tutorial 3, request es un objeto HttpRequest . Para más información sobre los objetos HttpRequest, consulte la documentación de petición y respuesta.

Después de que alguien vota en una pregunta, la vista vote() remite a la página de resultados de la pregunta. Vamos a escribir dicha vista:

In [ ]:

# polls/views.py

from django.shortcuts import get_object_or_404, render


def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, "polls/results.html", {"question": question})



Esto es casi exactamente lo mismo que la vista detail() del Tutorial 3. La única diferencia es el nombre de la plantilla. Solucionaremos esta redundancia más tarde.

Ahora, cree una plantilla polls/results.html:

A continuación, vaya a /polls/1/ en su navegador y vote en la pregunta. Debería ver una página de resultados que se actualiza cada vez que usted vota. Si usted envía el formulario sin haber seleccionado una opción, usted debería ver el mensaje de error.
## Utilice vistas genéricas: menos código es mejor

Las vistas de detalle() (del Tutorial 3) y resultados() son muy cortas y, como se mencionó anteriormente, redundantes. La vista index(), que muestra una lista de encuestas, es similar.

Estas vistas representan un caso común de desarrollo web básico: obtener datos de la base de datos de acuerdo con un parámetro pasado en la URL, cargar una plantilla y devolver la plantilla representada. Debido a que esto es tan común, Django proporciona un atajo, llamado sistema de «vistas genéricas».

Las vistas genéricas abstraen patrones comunes hasta el punto en que ni siquiera es necesario escribir código Python para escribir una aplicación. Por ejemplo, las vistas genéricas ListView y DetailView abstraen los conceptos de "mostrar una lista de objetos" y "mostrar una página de detalles para un tipo particular de objeto", respectivamente.

Convirtamos nuestra aplicación de encuesta para que utilice el sistema de vistas genéricas, de modo que podamos eliminar un montón de nuestro propio código. Tendremos que seguir algunos pasos para realizar la conversión. Lo haremos:

    Convertir el URLconf.
    Elimina algunas de las vistas viejas e innecesarias.
    Introduce nuevas vistas basadas en las vistas genéricas de Django.

Siga leyendo para conocer más detalles.

### ¿Por qué el intercambio de código?

Generalmente, cuando se escribe una aplicación de Django, usted podrá evaluar si las vistas genéricas son una buena opción para su problema y ​​las utilizará desde el principio en lugar de reestructurar su código a medio camino. Sin embargo, este tutorial intencionadamente se ha centrado hasta ahora en escribir las vistas «de la manera difícil» para centrarse en los conceptos esenciales.

Debe saber matemáticas básicas antes de comenzar a utilizar una calculadora.
## Modifique el URLconf

Primero, abra el URLconf polls/urls.py y modifíquelo de la siguiente manera:

In [ ]:

# polls/urls.py

from django.urls import path

from . import views

app_name = "polls"
urlpatterns = [
    path("", views.IndexView.as_view(), name="index"),
    path("<int:pk>/", views.DetailView.as_view(), name="detail"),
    path("<int:pk>/results/", views.ResultsView.as_view(), name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
]



Tenga en cuenta que el nombre del patrón coincidente en las cadenas de ruta del segundo y tercer patrón ha cambiado de <question_id> a <pk>. Esto es necesario porque usaremos la vista genérica DetailView para reemplazar nuestras vistas de detalles() y resultados(), y espera que el valor de clave principal capturado de la URL se llame "pk".
   

## Modifique las vistas

A continuación, vamos a eliminar nuestras viejas vistas index, Detail y Results y en su lugar vamos a usar las vistas genéricas de Django. Para ello, abra el archivo polls/views.py y modifíquelo de la siguiente manera:

In [ ]:

# polls/views.py

from django.db.models import F
from django.http import HttpResponseRedirect
from django.shortcuts import get_object_or_404, render
from django.urls import reverse
from django.views import generic

from .models import Choice, Question


class IndexView(generic.ListView):
    template_name = "polls/index.html"
    context_object_name = "latest_question_list"

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by("-pub_date")[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = "polls/detail.html"


class ResultsView(generic.DetailView):
    model = Question
    template_name = "polls/results.html"


def vote(request, question_id):
    # same as above, no changes needed.
    ...



Cada vista genérica necesita saber sobre qué modelo actuará. Esto se proporciona utilizando el atributo del modelo (en este ejemplo, modelo = Pregunta para DetailView y ResultsView) o definiendo el método get_queryset() (como se muestra en IndexView).

Por defecto, la vista genérica DetailView utiliza una plantilla llamada <app name>/<model name>_detail.html. En nuestro caso, utilizaría la plantilla "polls/question_detail.html". El atributo template_name se utiliza para indicarle a Django que utilice un nombre de plantilla específico en vez del nombre de plantilla generado de forma automática. También especificamos el atributo template_name para la vista de lista de resultados, esto garantiza que la vista de resultados y la vista detalle tengan un aspecto diferente cuando sean creadas, a pesar de que las dos son una vista genérica DetailView en segundo plano.

Del mismo modo, la vista genérica ListView utiliza una plantilla predeterminada llamada <app name>/<model name>_list.html; Utilizamos el atributo template_name para indicarle a ListView que utilice nuestra plantilla "polls/index.html" existente.

En partes anteriores del tutorial, a las plantillas se les proporcionó un contexto que contiene las variables de contexto question y Latest_question_list. Para DetailView, la variable de pregunta se proporciona automáticamente; dado que estamos usando un modelo de Django (Pregunta), Django puede determinar un nombre apropiado para la variable de contexto. Sin embargo, para ListView, la variable de contexto generada automáticamente es question_list. Para anular esto, proporcionamos el atributo context_object_name, especificando que queremos usar Latest_question_list en su lugar. Como enfoque alternativo, podrías cambiar tus plantillas para que coincidan con las nuevas variables de contexto predeterminadas, pero es mucho más fácil decirle a Django que use la variable que deseas.

Ejecute el servidor y utilice su nueva aplicación de encuesta basada en las vistas genéricas.

Para más detalles sobre las vistas genéricas, consulte la documentación de las vistas genéricas https://docs.djangoproject.com/es/5.0/topics/class-based-views/.